# Matrix operations

As linear algebra is fundamental in almost everything uses mathematics, matrices are everywhere in numerical analysis.  There isn't shortage of linear algebraic software packages and it's critically important to understand how they work.

1. POD arrays and majoring
2. Matrix-vector and matrix-matrix operations
3. Linear algebra

# POD arrays

The plain-old-data (POD) arrays are also called C-style arrays.  They are given the names because they are nothing more than just data and support no mechanism fancier than arithmetic.  We DO, oftentimes, wrap POD with fancy C++ constructs, but all the heavy-lifting numerical calculations still need to be done with POD, because essentially that's how von Neumann computers work.  (It clearly reveals itself in the machine code.  Check it yourselves.)

## Vector: 1D array

A vector is stored as a (usually contiguous) memory buffer of sequetially ordered elements.

In [ ]:
!make pod_vector; ./pod_vector

BLAS (basic linear albegra subprograms; a standard set of array manipulation API) defines vector operations as the 1st level.  A partial list of them:

* `SAXPY`: $\mathbf{y} = a\mathbf{x} + \mathbf{y}$, constant times a vector plus a vector
* `SDOT`: $\mathbf{x}\cdot\mathbf{y}$, dot product of two vectors.
* `SNRM2`: $\sqrt{\mathbf{y}\cdot\mathbf{y}}, $Euclidean norm.

The operations are simple enough that we usually don't need to call the library.

## Matrix: 2D array

See how to represent a $5\times5$ square matrix:

\begin{align*}
\mathrm{A} = \left[ a_{ij} \right] = \left(\begin{array}{ccccc}
a_{11} & a_{12} & a_{13} & a_{14} & a_{15} \\
a_{21} & a_{22} & a_{23} & a_{24} & a_{25} \\
a_{31} & a_{32} & a_{33} & a_{34} & a_{35} \\
a_{41} & a_{42} & a_{43} & a_{44} & a_{45} \\
a_{51} & a_{52} & a_{53} & a_{54} & a_{55}
\end{array}\right)
\end{align*}

$i$ is the row index (in the horizontal direction).  $j$ is the column index (in the vertical direction).  If we want to use the 0-based index, it can be rewritten as:

\begin{align*}
\mathrm{A} = \left[ a_{ij} \right] = \left(\begin{array}{ccccc}
a_{00} & a_{01} & a_{02} & a_{03} & a_{04} \\
a_{10} & a_{11} & a_{12} & a_{13} & a_{14} \\
a_{20} & a_{21} & a_{22} & a_{23} & a_{24} \\
a_{30} & a_{31} & a_{32} & a_{33} & a_{34} \\
a_{40} & a_{41} & a_{42} & a_{43} & a_{44}
\end{array}\right)
\end{align*}

In C++ we can use an auto variable like below for the matrix:

```cpp
constexpr size_t width = 5;

double amatrix[width][width];

// Populate the matrix on stack (row-major 2D array).
for (size_t i=0; i<width; ++i) // the i-th row
{
    for (size_t j=0; j<width; ++j) // the j-th column
    {
        amatrix[i][j] = i*10 + j;
    }
}

std::cout << "2D array elements:";
for (size_t i=0; i<width; ++i)
{
    std::cout << std::endl << " ";
    for (size_t j=0; j<width; ++j)
    {
        std::cout << " " << std::setfill('0') << std::setw(2)
                  << amatrix[i][j];
    }
}
std::cout << std::endl;
```

In [ ]:
!make pod_matrix_auto; ./pod_matrix_auto

The C++ multi-dimensional array index is convenient, but it doesn't work when the array size isn't known in the compile time.

```cpp
void work(double * buffer, size_t width)
{
    // This won't work as width isn't known in compile time.
    double (*matrix)[width] = reinterpret_cast<double (*)[width]>(buffer);

    for (size_t i=0; i<width; ++i) // the i-th row
    {
        for (size_t j=0; j<width; ++j) // the j-th column
        {
            matrix[i][j] = i*10 + j;
        }
    }

    std::cout << "matrix:";
    for (size_t i=0; i<width; ++i)
    {
        std::cout << std::endl << " ";
        for (size_t j=0; j<width; ++j)
        {
            std::cout << " " << std::setfill('0') << std::setw(2)
                      << matrix[i][j];
        }
    }
    std::cout << std::endl;
}
```

In [ ]:
!make pod_bad_matrix

## Row-major 2D variable-size array

The elements of a row-major 2D array are stored so that the fastest changing index is the trailing index of the 2D array:

\begin{align*}
\mathrm{buffer} = [a_{00}, a_{01}, a_{02}, a_{03}, a_{04}, a_{10}, a_{11}, a_{12}, \ldots, a_{43}, a_{44}]
\end{align*}

When accessing the elements, all we need to do is to remember how long we need to *stride* per row (leading) index.

```cpp
constexpr size_t width = 5;

double * buffer = new double[width*width];
double (*matrix)[width] = reinterpret_cast<double (*)[width]>(buffer);
std::cout << "buffer address: " << buffer << std::endl
          << "matrix address: " << matrix << std::endl;

// Populate a buffer (row-major 2D array).
for (size_t i=0; i<width; ++i) // the i-th row
{
    for (size_t j=0; j<width; ++j) // the j-th column
    {
        buffer[i*width + j] = i*10 + j;
    }
}

std::cout << "matrix (row-major) elements as 2D array:";
for (size_t i=0; i<width; ++i)
{
    std::cout << std::endl << " ";
    for (size_t j=0; j<width; ++j)
    {
        std::cout << " " << std::setfill('0') << std::setw(2)
                  << matrix[i][j];
    }
}
std::cout << std::endl;
```

In [ ]:
!make pod_matrix_rowmajor; ./pod_matrix_rowmajor

## Column-major 2D variable-size array

The elements of a column-major 2D array are stored so that the fastest changing index is the leading index of the 2D array:

\begin{align*}
\mathrm{buffer} = [a_{00}, a_{10}, a_{20}, a_{30}, a_{40}, a_{01}, a_{11}, a_{21}, \ldots, a_{34}, a_{44}]
\end{align*}

Similar to a row-major array, we need to know the stride.  But this time it's for the column (trailing) index.

```cpp
constexpr size_t width = 5;

double * buffer = new double[width*width];
double (*matrix)[width] = reinterpret_cast<double (*)[width]>(buffer);
std::cout << "buffer address: " << buffer << std::endl
          << "matrix address: " << matrix << std::endl;

// Populate a buffer (column-major 2D array).
for (size_t i=0; i<width; ++i) // the i-th row
{
    for (size_t j=0; j<width; ++j) // the j-th column
    {
        buffer[j*width + i] = i*10 + j;
    }
}

std::cout << "matrix (column-major) elements as 2D array:";
for (size_t i=0; i<width; ++i)
{
    std::cout << std::endl << " ";
    for (size_t j=0; j<width; ++j)
    {
        std::cout << " " << std::setfill('0') << std::setw(2)
                  << matrix[j][i];
    }
}
std::cout << std::endl;
```

In [ ]:
!make pod_matrix_colmajor; ./pod_matrix_colmajor

## C++ class to treat the memory buffer like matrix

Keeping track of the stride can be error-prone.  Even if we stick to one majoring order (usually it's row-majoring), it's easy to lose track of it when the number of row and column is different, or it's higher-dimensional.

A common practice in C++ is to use a class to keep track of the stride.  Properly defined accessors significantly simplifies it.

```cpp
class Matrix {

public:

    Matrix(size_t nrow, size_t ncol)
      : m_nrow(nrow), m_ncol(ncol)
    {
        size_t nelement = nrow * ncol;
        m_buffer = new double[nelement];
    }

    // TODO: copy and move constructors and assignment operators.

    ~Matrix()
    {
        delete[] m_buffer;
    }

    // No bound check.
    double   operator() (size_t row, size_t col) const { return m_buffer[row*m_ncol + col]; }
    double & operator() (size_t row, size_t col)       { return m_buffer[row*m_ncol + col]; }

    size_t nrow() const { return m_nrow; }
    size_t ncol() const { return m_ncol; }

private:

    size_t m_nrow;
    size_t m_ncol;
    double * m_buffer;

};
```

In [ ]:
!make matrix_class; ./matrix_class

## Matrix-vector operation

Operations of a matrix and a vector is much more interesting than vector operations.

# Matrix-matrix operation

# Linear algebra

## Least square

## Eigen problems and SVD

# References

* BLAS: http://www.netlib.org/blas/

# Exercises

1. Extend the class `Matrix` to be an arbitrary dimensional array.